In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
import kagglehub
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\kujoo\.cache\kagglehub\datasets\prajitdatta\movielens-100k-dataset\versions\1


In [6]:
ratings_path = path + "/ml-100k/u.data"
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_path, sep='\t', names=columns)
print("Ratings DataFrame shape:", ratings.shape)
ratings.head()

Ratings DataFrame shape: (100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
user_item_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')
user_item_matrix = user_item_matrix.fillna(0)
print("User-Item Matrix shape:", user_item_matrix.shape)
user_item_matrix.head()

User-Item Matrix shape: (943, 1682)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print("User Similarity Matrix shape:", user_similarity_df.shape)
user_similarity_df.head()

User Similarity Matrix shape: (943, 943)


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941


In [10]:
def get_unseen_movies(user_id, rating_df, all_movie_ids):
    seen = set(ratings.loc[ratings['user_id'] == user_id, 'movie_id'])
    return [m for m in all_movie_ids if m not in seen]
all_moivie_ids = user_item_matrix.columns.tolist()

In [11]:
def recommend_for_user(user_id, k_neighbors=25, top_n=10):
    sims = user_similarity_df.loc[user_id].drop(user_id)
    neighbors = sims.nlargest(k_neighbors)
    seen = set(ratings.loc[ratings['user_id'] == user_id, 'movie_id'])
    unseen = [m for m in user_item_matrix.columns if m not in seen]
    if not unseen:
        return []
    scores = {}
    
    for m in unseen:
        r = user_item_matrix.loc[neighbors.index, m]
        mask = r > 0
        if mask.any():
            scores[m] = np.dot(r[mask], neighbors[mask]) / neighbors[mask].sum()
        else:
            scores[m] = 0.0
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]

user_id_example = user_item_matrix.index[0]
recommend_for_user(user_id_example, k_neighbors=25, top_n=10)


[(285, np.float64(5.0)),
 (489, np.float64(5.0)),
 (492, np.float64(5.0)),
 (602, np.float64(5.0)),
 (641, np.float64(5.0)),
 (690, np.float64(5.0)),
 (853, np.float64(5.0)),
 (1009, np.float64(5.0)),
 (1143, np.float64(5.0)),
 (1168, np.float64(5.0))]